<a href="https://colab.research.google.com/github/varunjagini/DRONE-MODULE/blob/main/lorawan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import math

In [4]:
def estimate_L_C(x, y):
    ''' 
    x = List of known meanRSSI value
    y = log(d) values corresponding to the meanRSSI values
    Output = L, C
    '''

    num = sum([a*b for a,b in zip(x,y)])  - 5*np.mean(x)*np.mean(y)
    den = sum([a*a - 5*(a*a) for a in x])

    slope = num/den

    L = -slope/10
    C = slope*np.mean(x) - np.mean(y)

    return L, C 

In [5]:
def estimate_SD(GD, H, beta):
    
    SD = np.sqrt(GD*GD + H*H - (2*GD*H*math.cos(beta)))

    return SD

In [6]:
def estimate_constants():
    ''' 
    In this function we will get the known meanRSSI values. 
    You have to get it from the input. 
    We assume we are getting 10 meanRSSI values and stored in the variable mRSSI
    We also assume we know GD and beta for each 10 mRSSI values
    '''
    # mRSSI, GD, beta
    mRSSI = [1.0, 2.2, 3.6, 5.3, 2.6, 2.7, 1.2, 5.8, 3.6, 10.0]
    GD = 20 
    beta = 90
    H = 50

    d = [estimate_SD(GD, H, beta) for _ in range(len(mRSSI))]

    L, C = estimate_L_C(mRSSI, list(map(math.log10, d)))
    
    return  d, L, C

In [11]:
def estimate_location():
    '''
    Here we get mean RSSI values from 4 GSs with locations (x1, y1, z1), (x2, y2, z2), (x3, y3, z3), (x4, y4, z4)
    '''
    
    r = []
    RSSI = [4.9, 1.2, 3.5, 2.1]
    (x1,y1,z1) = (1,3,4)
    (x2,y2,z2) = (2,8,6)
    (x3,y3,z3) = (1,3,6)
    (x4,y4,z4) = (1,6,2)
    for i in range(4):
        d, L, C = estimate_constants()
        r.append(10**(-((RSSI[i]-C)/10*L)))
    
    A = [2*(np.array([x4-x1, y4-y1, z4-z1])), 2*(np.array([x4-x2, y4-y2, z4-z2])), 2*(np.array([x4-x3, y4-y3, z4-z3]))]
    B = [[r[0]**2 - r[3]**2 - x1**2 - y1**2 - z1**2 + x4**2 + y4**2 + z4**2], [r[1]**2 - r[3]**2 - x2**2 - y2**2 - z2**2 + x4**2 + y4**2 + z4**2], [r[2]**2 - r[3]**2 - x3**2 - y3**2 - z3**2 + x4**2 + y4**2 + z4**2]]

    AT =  np.array(A).T.tolist()

    t1 = np.linalg.inv(np.matmul(AT, A))
    t2 = np.matmul(t1, AT)
    w = np.matmul(t2,B)

    return w

In [12]:
print(estimate_location())

[[-0.16231651]
 [ 5.83203589]
 [ 4.99935185]]
